In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings("ignore")

from Faster_RCNN.Train import Train_Faster_RCNN
from Faster_RCNN.Test import Test_Faster_RCNN
from Faster_RCNN.Valid import Valid_Faster_RCNN
from Faster_RCNN.Toolbox import check_dataset

seed= 1
random.seed(seed)
np.random.seed(seed)

# Prepare Data

In [ ]:
import json

all_dataset= []

with open('./Data/train_1.json', 'r', encoding="utf-8") as f:
    label= json.load(f)  
for i in tqdm(range(len(label['images']))):
    data= {}
    data['image_path']= './Data/train_img_1/' + label['images'][i]['file_name']
    data['bbox']= []
    data['label']= []
    id= label['images'][i]['id']
    
    for j in range(len(label['annotations'])):
        if id==label['annotations'][j]['image_id']:
            box= label['annotations'][j]['bbox']
            box[2]+= box[0]
            box[3]+= box[1]
            data['bbox'].append(box)
            data['label'].append(label['annotations'][j]['category_id'])
    all_dataset.append(data)
    
# with open('./Data/train_2.json', 'r', encoding="utf-8") as f:
#     label= json.load(f)  
# for i in tqdm(range(len(label['images']))):
#     data= {}
#     data['image_path']= './Data/train_img_2/' + label['images'][i]['file_name']
#     data['bbox']= []
#     data['label']= []
#     id= label['images'][i]['id']
    
#     for j in range(len(label['annotations'])):
#         if id==label['annotations'][j]['image_id']:
#             box= label['annotations'][j]['bbox']
#             box[2]+= box[0]
#             box[3]+= box[1]
#             data['bbox'].append(box)
#             data['label'].append(label['annotations'][j]['category_id'])
#     all_dataset.append(data)
    
            
all_dataset= check_dataset(all_dataset[:])
all_dataset[:3]

# 過濾標籤類別

In [ ]:
keep_classes= [1,2,3,4,5]

drop_sample= []
for i, data in enumerate(tqdm(all_dataset)):
    
    bbox= data['bbox']
    label= data['label']
    
    drop_indx= []
    for j in range(len(label)):
        if label[j] not in keep_classes:
            drop_indx.append(j)
            
    data['bbox']= np.delete(bbox, drop_indx, axis= 0)
    data['label']= np.delete(label, drop_indx, axis= 0)
    
    if len(data['bbox'])==0: drop_sample.append(i)

all_dataset= np.delete(all_dataset, drop_sample, axis= 0)

print('drop empty sample: {}'.format(len(drop_sample)))

# 將具有大量bbox的資料移至validation

In [ ]:
import matplotlib.pyplot as plt

drop_indx= []
for i, data in enumerate(tqdm(all_dataset)):
    
    #img= cv2.imread(data['image_path'])
    bbox= data['bbox']
    bbox= np.array(bbox).astype(np.int)

    if len(bbox)>100:
        drop_indx.append(i)
        continue
        print(len(bbox))
        for box in bbox:
            cv2.rectangle(img,
                          (box[0], box[1]),
                          (box[2], box[3]),
                          (255, 0, 0), 10)
        plt.imshow(img)
        plt.show()

vali_dataset= np.array(all_dataset)[drop_indx]
train_dataset= np.delete(all_dataset, drop_indx, axis= 0)
print('move {} sample to validation'.format(len(drop_indx)))

# Valid

In [ ]:
import torch

Valid_CFG= {
    'img_size': None,    # None為使用原始解析度
    'confidence': 0.001,
    'NMS_threshold': 0.5,
    'load_model': 'train_cv_model/faster_rcnn_best.pth',
    'device': 'gpu:1',
}

valid_IOU, valid_class_IOU, MAP, AP_class= Valid_Faster_RCNN(
                                                    dataset= train_dataset[:],
                                                    CFG= Valid_CFG,
                                                )

# Test

In [ ]:
Test_CFG= {
    'img_size': None, # None為使用原始解析度
    'confidence': 0.001,
    'NMS_threshold': 0.5,
    'load_model': 'test_cv_model/resnet152_1536_0.399/faster_rcnn_best.pth',
    'show_img': False,
    'show_score': False,
    'show_classes': False,
    'save_img_path': False, #'test/predict', # False則不儲存結果圖片
    'device': 'gpu:0',
}

result= Test_Faster_RCNN(
            dataset= train_dataset,
            CFG= Test_CFG,
        )

In [ ]:
import matplotlib.pyplot as plt

color= [
    (255, 0, 0), #紅
    (0, 255, 0), #綠
    (0, 0, 255), #藍
    (255, 97, 0), #澄
    (255, 0, 255), #紫
]

drop_name= []
for i in range(len(all_map)):

    if all_map[i] > 0.4: continue
    
    for data in tqdm(train_dataset[i:i+1]):

        name= data['image_path']
        drop_name.append(name)
        img= cv2.imread(name)
        img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label= data['label']
        bbox= data['bbox']
        bbox= np.array(bbox).astype(np.int)

        for k, box in enumerate(bbox):
            cv2.rectangle(img,
                          (box[0], box[1]),
                          (box[2], box[3]),
                          color[label[k]-1], 10)

        plt.title('true label')
        plt.imshow(img)
        plt.show()

    for data in tqdm(result[i:i+1]):

        name= data['img_name']
        print('map:', all_map[i])
        img= cv2.imread(name)
        img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label= data['label']
        bbox= data['bbox']
        bbox= np.array(bbox).astype(np.int)
        score= data['score']

        for k, box in enumerate(bbox):
            if score[k]>=0.001:
                cv2.rectangle(img,
                              (box[0], box[1]),
                              (box[2], box[3]),
                              color[label[k]-1], 10)

        plt.title('predict')
        plt.imshow(img)
        plt.show()